# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from pathlib import Path
import re
import numpy as np
import json
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words

In [4]:
# load data from database
repo = Path.cwd()
path = repo / 'data' / 'DisasterResponse.db'

engine = create_engine(f'sqlite:///{path}')
df = pd.read_sql("select * from Message", con=engine)
df.shape

(25991, 38)

#check imbalance in target
See explore_data.ipynb

### 2. Write a tokenization function to process your text data

In [5]:
stop_words = stopwords.words("english")
vocab = set(words.words())
vocab = vocab - set(stop_words)
lemmatizer = WordNetLemmatizer()
words_set = {lemmatizer.lemmatize(word) for word in vocab}

In [6]:
def tokenize(text):
    """TO DO
    """
    
    text = re.sub(r"[^a-zA-Z]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = [lemmatizer.lemmatize(token).strip() for token in tokens if token in words_set]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [37]:
pipeline = Pipeline([
    ("cv", CountVectorizer(tokenizer=tokenize)),
    ("transformer", TfidfTransformer()),
    ("ada", MultiOutputClassifier(AdaBoostClassifier(n_estimators=100)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [38]:

X = df['message'].values
Y = df.iloc[:, :-3].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((20792,), (5199,), (20792, 35), (5199, 35))

In [39]:
pipeline.fit(X_train, Y_train)


Pipeline(steps=[('cv',
                 CountVectorizer(tokenizer=<function tokenize at 0x1209953a0>)),
                ('transformer', TfidfTransformer()),
                ('ada',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(n_estimators=100)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [40]:
Y_preds = pipeline.predict(X_test)
output_labels = list(df.iloc[:, :-3].columns)

In [41]:
def get_metrics(y_test, y_pred):
    """TO DO
    """
    cr = classification_report(y_test, y_pred, zero_division=0, output_dict=True)
    return cr

In [42]:
def generate_report():
    report_dict = {}
    for idx, output in enumerate(output_labels):
        metrics_dict = get_metrics(Y_test[:,idx], Y_preds[:,idx])
        report_dict[output] = metrics_dict
        
    with open('ada_model_results.json', 'w', encoding="utf-8") as fhand:
        json.dump(report_dict, fhand)
    
    for k, v in report_dict.items():
        print(f"""category: {k} \n accuracy: {v['accuracy']:.4f}, precision: {v['1']['precision']:.4f}, recall: {v['1']['recall']:.4f}, f-score: {v['1']['f1-score']:.4f}""")

generate_report()

category: related 
 accuracy: 0.8000, precision: 0.8137, recall: 0.9592, f-score: 0.8805
category: request 
 accuracy: 0.8821, precision: 0.7603, recall: 0.4957, f-score: 0.6001
category: offer 
 accuracy: 0.9942, precision: 0.0000, recall: 0.0000, f-score: 0.0000
category: aid_related 
 accuracy: 0.7403, precision: 0.7392, recall: 0.5879, f-score: 0.6549
category: medical_help 
 accuracy: 0.9254, precision: 0.6429, recall: 0.2838, f-score: 0.3937
category: medical_products 
 accuracy: 0.9556, precision: 0.5701, recall: 0.2480, f-score: 0.3456
category: search_and_rescue 
 accuracy: 0.9731, precision: 0.4750, recall: 0.1377, f-score: 0.2135
category: security 
 accuracy: 0.9815, precision: 0.0833, recall: 0.0116, f-score: 0.0204
category: military 
 accuracy: 0.9648, precision: 0.4227, recall: 0.2440, f-score: 0.3094
category: water 
 accuracy: 0.9644, precision: 0.7188, recall: 0.6656, f-score: 0.6912
category: food 
 accuracy: 0.9452, precision: 0.8392, recall: 0.6425, f-score: 0.727

### 6. Improve your model
Use grid search to find better parameters. 

In [43]:
pipeline.get_params()

{'memory': None,
 'steps': [('cv',
   CountVectorizer(tokenizer=<function tokenize at 0x1209953a0>)),
  ('transformer', TfidfTransformer()),
  ('ada',
   MultiOutputClassifier(estimator=AdaBoostClassifier(n_estimators=100)))],
 'verbose': False,
 'cv': CountVectorizer(tokenizer=<function tokenize at 0x1209953a0>),
 'transformer': TfidfTransformer(),
 'ada': MultiOutputClassifier(estimator=AdaBoostClassifier(n_estimators=100)),
 'cv__analyzer': 'word',
 'cv__binary': False,
 'cv__decode_error': 'strict',
 'cv__dtype': numpy.int64,
 'cv__encoding': 'utf-8',
 'cv__input': 'content',
 'cv__lowercase': True,
 'cv__max_df': 1.0,
 'cv__max_features': None,
 'cv__min_df': 1,
 'cv__ngram_range': (1, 1),
 'cv__preprocessor': None,
 'cv__stop_words': None,
 'cv__strip_accents': None,
 'cv__token_pattern': '(?u)\\b\\w\\w+\\b',
 'cv__tokenizer': <function __main__.tokenize(text)>,
 'cv__vocabulary': None,
 'transformer__norm': 'l2',
 'transformer__smooth_idf': True,
 'transformer__sublinear_tf': Fa

In [45]:
type(pipeline)

sklearn.pipeline.Pipeline

In [24]:
from sklearn.model_selection import GridSearchCV

In [27]:
pipeline = Pipeline([
    ("cv", CountVectorizer(tokenizer=tokenize)),
    ("transformer", TfidfTransformer()),
    ("ada", MultiOutputClassifier(AdaBoostClassifier(n_estimators=100, random_state=42)))
])

param_grid = {
    "cv__max_df": (0.5, 1.0),
    "ada__estimator__learning_rate": [0.5, 1.0],
}

cv = GridSearchCV(pipeline, param_grid=param_grid, scoring="f1_micro")

In [28]:
cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('cv',
                                        CountVectorizer(tokenizer=<function tokenize at 0x1209953a0>)),
                                       ('transformer', TfidfTransformer()),
                                       ('ada',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier(n_estimators=100,
                                                                                           random_state=42)))]),
             param_grid={'ada__estimator__learning_rate': [0.5, 1.0],
                         'cv__max_df': (0.5, 1.0)},
             scoring='f1_micro')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
cv.best_params_

{'ada__estimator__learning_rate': 1.0, 'cv__max_df': 0.5}

In [30]:
Y_preds = cv.predict(X_test)

In [35]:
def generate_report():
    report_dict = {}
    for idx, output in enumerate(output_labels):
        metrics_dict = get_metrics(Y_test[:,idx], Y_preds[:,idx])
        report_dict[output] = metrics_dict
        
    with open('ada_gridsearchcv_results.json', 'w', encoding="utf-8") as fhand:
        json.dump(report_dict, fhand)
    
    for k, v in report_dict.items():
        print(f"""category: {k} \n accuracy: {v['accuracy']:.4f}, precision: {v['1']['precision']:.4f}, recall: {v['1']['recall']:.4f}, f-score: {v['1']['f1-score']:.4f}""")

In [36]:
generate_report()

category: related 
 accuracy: 0.8000, precision: 0.8137, recall: 0.9592, f-score: 0.8805
category: request 
 accuracy: 0.8821, precision: 0.7603, recall: 0.4957, f-score: 0.6001
category: offer 
 accuracy: 0.9940, precision: 0.0000, recall: 0.0000, f-score: 0.0000
category: aid_related 
 accuracy: 0.7403, precision: 0.7392, recall: 0.5879, f-score: 0.6549
category: medical_help 
 accuracy: 0.9254, precision: 0.6429, recall: 0.2838, f-score: 0.3937
category: medical_products 
 accuracy: 0.9556, precision: 0.5701, recall: 0.2480, f-score: 0.3456
category: search_and_rescue 
 accuracy: 0.9731, precision: 0.4750, recall: 0.1377, f-score: 0.2135
category: security 
 accuracy: 0.9815, precision: 0.0833, recall: 0.0116, f-score: 0.0204
category: military 
 accuracy: 0.9648, precision: 0.4227, recall: 0.2440, f-score: 0.3094
category: water 
 accuracy: 0.9646, precision: 0.7213, recall: 0.6656, f-score: 0.6923
category: food 
 accuracy: 0.9452, precision: 0.8392, recall: 0.6425, f-score: 0.727

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [44]:
import pickle

pickle.dump(cv, open('model_ada_gridsearchcv.pkl','wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.